In [1]:
print("Hello, world!")

Hello, world!


In [7]:
import csv
from faker import Faker
import random



In [8]:
fake = Faker()

In [9]:
categories = ["Electronics", "Clothing", "Home & Furniture", "Grocery", "Sports"]

In [10]:
products = [
 {"id":"A01","name": "Mobile", "price": 35000, "category": "Electronics"},
 {"id":"A02","name": "TV", "price": 65000, "category": "Electronics"},
 {"id":"A03","name": "Headphone", "price": 2000, "category": "Electronics"},
 {"id":"A04","name": "Laptop", "price": 95000, "category": "Electronics"},
 {"id":"A05","name": "Watch", "price": 1500, "category": "Electronics"},
 {"id":"B01","name": "Jeans", "price": 999, "category": "Clothing"},
 {"id":"B02","name": "T-shirt", "price": 799, "category": "Clothing"},
 {"id":"B03","name": "Kurta", "price": 1999, "category": "Clothing"},
 {"id":"B04","name": "Saree", "price": 2449, "category": "Clothing"},
 {"id":"B05","name": "Belt", "price": 499, "category": "Clothing"},
 {"id":"C01","name": "Bedsheet", "price": 1999, "category": "Home & Furniture"},
 {"id":"C02","name": "Mattres", "price": 999, "category": "Home & Furniture"},
 {"id":"C03","name": "Chair", "price": 899, "category": "Home & Furniture"},
 {"id":"C04","name": "Gas-stove", "price": 599, "category": "Home & Furniture"},
 {"id":"C05","name": "Wall Clock", "price": 999, "category": "Home & Furniture"},
 {"id":"D01","name": "Oil-Ghee", "price": 499, "category": "Grocery"},
 {"id":"D02","name": "Detergent", "price": 299, "category": "Grocery"},
 {"id":"D03","name": "Snacks", "price": 199, "category": "Grocery"},
 {"id":"D04","name": "Salt", "price": 199, "category": "Grocery"},
 {"id":"D05","name": "Atta", "price": 699, "category": "Grocery"},
 {"id":"E01","name": "Tracker", "price": 2999, "category": "Sports"},
 {"id":"E02","name": "Yoga-Mat", "price": 1599, "category": "Sports"},
 {"id":"E03","name": "Jump-rope", "price": 399, "category": "Sports"},
 {"id":"E04","name": "Band", "price": 1499, "category": "Sports"},
 {"id":"E05","name": "Shoes", "price": 2999, "category": "Sports"},
 ]

In [11]:
# Define the users
users = []
for i in range(50):
    name = fake.name()
 
    email = name.lower().replace(" ", ".") + "@example.com"
    users.append({"id": i+1,"name": name, "email": email})

In [12]:
# Define the purchased, rated, and disrated relationships between users and products
purchased = []
rated = []
disrated = []
for user in users:
    user_products = random.sample(products, random.randint(1, len(products)))
    for product in user_products:
        if random.random() < 0.6: # 60% chance of purchasing
            purchased.append({"user": user["id"], "product":
            product["id"]})
        elif random.random() < 0.5: # 50% chance of liking
            rated.append({"user": user["id"], "product": product["id"],
            "rating":random.random()*5})

In [13]:
 # Write the data to CSV files
with open("users.csv", "w", newline="") as csvfile:
    fieldnames = ["id", "name", "email"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for i, user in enumerate(users):
        writer.writerow({
            "id": user["id"], "name": user["name"],
            "email":user["email"]
        })

In [14]:
with open("products.csv", "w", newline="") as csvfile:
    fieldnames = ["id", "name", "price", "category"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for i, product in enumerate(products):
        writer.writerow({"id": product["id"], "name": product["name"],
        "price": product["price"], "category": product["category"]})

In [15]:
with open("purchased.csv", "w", newline="") as csvfile:
    fieldnames = ["user", "product"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for purchase in purchased:
        writer.writerow({"user": purchase["user"], "product": purchase["product"]})

In [16]:
with open("rated.csv", "w", newline="") as csvfile:
    fieldnames = ["user", "product","rating"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for rate in rated:
        writer.writerow({
        "user": rate["user"],
        "product": rate["product"],
        "rating":rate["rating"]
        })

In [2]:
from neo4j import GraphDatabase

uri = "neo4j://127.0.0.1:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "@Simran1234"))

In [4]:
import pandas as pd

df_users = pd.read_csv("users.csv")

def create_user(tx, user_id, name, email):
    tx.run("CREATE (:User {id: $user_id, name: $name, email: $email})",
           user_id=user_id, name=name, email=email)

with driver.session() as session:
    for _, row in df_users.iterrows():
        session.execute_write(create_user, row['id'], row['name'], row['email'])

print("All users imported to Neo4j!")

All users imported to Neo4j!


In [11]:
import pandas as pd

# Load purchased-checkpoint CSV
df_purchased = pd.read_csv("/Users/simran/Documents/pr/Graph-based-Recommendation-System/.ipynb_checkpoints/purchased-checkpoint.csv")

# Function to create PURCHASED relationships
def create_purchase(tx, user_id, product_id):
    tx.run(
        """
        MATCH (u:User {id: $user_id})
        MATCH (p:Product {id: $product_id})
        CREATE (u)-[:PURCHASED]->(p)
        """,
        user_id=user_id, product_id=product_id
    )

# Execute creation in Neo4j
with driver.session() as session:
    for _, row in df_purchased.iterrows():
        session.execute_write(create_purchase, row['user'], row['product'])

print("All purchases imported to Neo4j!")


All purchases imported to Neo4j!


In [13]:
import pandas as pd
from neo4j import GraphDatabase

# Neo4j connection
uri = "neo4j://127.0.0.1:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "@Simran1234"))

# Load the CSV
df_rated = pd.read_csv("rated.csv")  # columns: user, product, rating

# Function to create RATED relationships
def create_rating(tx, user, product, rating):
    tx.run("""
        MATCH (u:User {id: $user}), (p:Product {id: $product})
        CREATE (u)-[:RATED {rating: $rating}]->(p)
    """, user=user, product=product, rating=rating)

# Execute for all rows
with driver.session() as session:
    for _, row in df_rated.iterrows():
        session.execute_write(create_rating, row['user'], row['product'], row['rating'])

print("All ratings imported to Neo4j!")


All ratings imported to Neo4j!


In [16]:
import pandas as pd
from neo4j import GraphDatabase

# Neo4j connection
uri = "neo4j://127.0.0.1:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "@Simran1234"))

# Load friends CSV
df_friend = pd.read_csv("/Users/simran/Documents/pr/Graph-based-Recommendation-System/Dataset/friends.csv")  # columns: user1,user2

# Function to create FRIEND relationships
def create_friendship(tx, user1, user2):
    tx.run("""
        MATCH (u1:User {id: $user1}), (u2:User {id: $user2})
        MERGE (u1)-[:FRIEND]->(u2)
        MERGE (u2)-[:FRIEND]->(u1)
    """, user1=user1, user2=user2)

# Execute for all rows
with driver.session() as session:
    for _, row in df_friend.iterrows():
        session.execute_write(create_friendship, row['user1'], row['user2'])

print("All friendships imported to Neo4j!")


All friendships imported to Neo4j!


In [26]:
import pandas as pd
from neo4j import GraphDatabase

# Neo4j connection
uri = "neo4j://127.0.0.1:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "@Simran1234"))

# Load similar.csv
df_similar = pd.read_csv("/Users/simran/Documents/pr/Graph-based-Recommendation-System/Dataset/similar.csv")  # columns: product_1_id, product_2_id

# Function to create SIMILAR relationships
def create_similarity(tx, product1, product2):
    tx.run("""
        MATCH (p1:Product {id: $product1}), (p2:Product {id: $product2})
        MERGE (p1)-[:SIMILAR]->(p2)
        MERGE (p2)-[:SIMILAR]->(p1)
    """, product1=product1, product2=product2)

# Execute for all rows
with driver.session() as session:
    for _, row in df_similar.iterrows():
        session.execute_write(create_similarity, row['product_1_id'], row['product_2_id'])

print("All product similarities imported to Neo4j!")


All product similarities imported to Neo4j!


In [27]:
import pandas as pd

df_products = pd.read_csv("products.csv")

def create_product(tx, product_id, name, price, category):
    tx.run(
        "CREATE (:Product {id: $product_id, name: $name, price: $price, category: $category})",
        product_id=product_id, name=name, price=price, category=category
    )

with driver.session() as session:
    for _, row in df_products.iterrows():
        session.execute_write(create_product, row['id'], row['name'], row['price'], row['category'])

print("All products imported to Neo4j!")


All products imported to Neo4j!
